In [ ]:
import numpy as np
import pandas as pd
from ConfigSpace import ConfigurationSpace, Configuration
from smac import MultiFidelityFacade, Scenario
from functools import partial


from datasets import get_hits
from metrics.tracks import track_metrics
from segment.track import gen_seg_track_layered
from segment.candidate import gen_seg_layered
from hopfield.energy.cross import cross_energy_matrix
from hopfield.energy.curvature import find_consecutive_segments, curvature_energy_matrix
from hopfield.iterate import annealing_curve, anneal, update_act_bulk, update_act_sequential
from optimize import CONFIG_DEFAULTS


# Learning rate vs sequential update?
- small easy dataset
- many trials are needed to find a workable config

In [ ]:
from datasets import spdsim
# small easy dataset
N_EVENTS = 50
hits = get_hits('spdsim', n_events=N_EVENTS, n_noise_hits=10, event_size=20)
hits[['x', 'y', 'z']] /= spdsim.LAYER_DIST
events = {eid: event.reset_index(drop=True) for eid, event in hits.groupby('event_id')}
eids = tuple(events.keys())
segs = {eid: gen_seg_layered(event) for eid, event in events.items()}
pairs = {eid: find_consecutive_segments(seg) for eid, seg in segs.items()}
crosses = {eid: cross_energy_matrix(seg) for eid, seg in segs.items()}

# fast iteration to optimize faster
extra_conf = {
    'cooling_steps': 10,
    'rest_steps': 0,
}

## bulk update, no learning rate

In [ ]:
# need many trials to find any working config
N_TRIALS = 100

scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.),
    }),
    'bulk-norate',
    n_trials=N_TRIALS,
    min_budget=3,
    max_budget=N_EVENTS
)


In [ ]:
def evaluate(config: Configuration, seed: int, budget: float) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(extra_conf)
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(eids, int(budget), replace=False):
        event = events[eid]
        seg = segs[eid]
        crossing_matrix = conf['alpha'] * crosses[eid]
        curvature_matrix = curvature_energy_matrix(
            pos=event[['x', 'y', 'z']].to_numpy(),
            seg=seg, pairs=pairs[eid], alpha=conf['alpha'], gamma=conf['gamma'],
            cosine_threshold=conf['cosine_min_rewarded'], cosine_min_allowed=conf['cosine_min_allowed'],
            curvature_cosine_power=conf['cosine_power'],
            do_sum_r=conf['distance_op'] == 'sum', distance_prod_power_in_denominator=conf['distance_power']
        )
        energy_matrix = crossing_matrix + curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = partial(update_act_bulk, learning_rate=conf['learning_rate'])
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>conf['threshold'])
        score['total steps'] = conf['cooling_steps'] + conf['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']


In [ ]:
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()


In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

## bulk update, optimize learning rate

In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.),
        'learning_rate': (0., 1.)
    }),
    'bulk-rate',
    n_trials=N_TRIALS,
    min_budget=3,
    max_budget=N_EVENTS
)

In [ ]:
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()

In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)

## sequential update

In [ ]:
scenario = Scenario(
    ConfigurationSpace({
        'alpha': (0., 1000.),
        'gamma': (0., 2000.),
        'bias': (-40.0, 40.0),
        'cosine_power': (0.0, 50.0),
        'cosine_min_rewarded': (0., 1.),
        't_max': (1., 1000.),
        'initial_act': (0., 1.)
    }),
    'sequential',
    n_trials=N_TRIALS,
    min_budget=3,
    max_budget=N_EVENTS
)

In [ ]:
def evaluate(config: Configuration, seed: int, budget: float) -> float:
    conf = CONFIG_DEFAULTS.copy()
    conf.update(extra_conf)
    conf.update(config)
    rng = np.random.default_rng(seed=seed)
    scores = []
    for eid in rng.choice(eids, int(budget), replace=False):
        event = events[eid]
        seg = segs[eid]
        crossing_matrix = conf['alpha'] * crosses[eid]
        curvature_matrix = curvature_energy_matrix(
            pos=event[['x', 'y', 'z']].to_numpy(),
            seg=seg, pairs=pairs[eid], alpha=conf['alpha'], gamma=conf['gamma'],
            cosine_threshold=conf['cosine_min_rewarded'], cosine_min_allowed=conf['cosine_min_allowed'],
            curvature_cosine_power=conf['cosine_power'],
            do_sum_r=conf['distance_op'] == 'sum', distance_prod_power_in_denominator=conf['distance_power']
        )
        energy_matrix = crossing_matrix + curvature_matrix
        temp_curve = annealing_curve(conf['t_min'], conf['t_max'],
                                     conf['cooling_steps'], conf['rest_steps'])
        act = np.full(len(seg), conf['initial_act'])
        update_act = update_act_sequential
        for _ in anneal(energy_matrix, temp_curve, act, update_act, bias=conf['bias']):
            pass
        tseg = gen_seg_track_layered(event)
        score = track_metrics(event, seg, tseg, act, act>conf['threshold'])
        score['total steps'] = conf['cooling_steps'] + conf['rest_steps']
        score['trackml loss'] = 1. - score['trackml score']
        scores.append(score)
    return pd.DataFrame(scores).mean()['trackml loss']

In [ ]:
optimizer = MultiFidelityFacade(scenario, evaluate, overwrite=True)
best_config = optimizer.optimize()

In [ ]:
optimizer.validate(best_config)

In [ ]:
best_config

In [ ]:
pd.DataFrame(optimizer.intensifier.trajectory)